In [1]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
# from lightgbm import LGBMClassifier, LGBMRegressor
# from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]
Y = train['quality'] 

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]

# Optimal Rounder

In [3]:
from functools import partial
import numpy as np
import scipy as sp

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights = 'quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p

    def coefficients(self):
        return self.coef_['x']

# Optuna Optimization

In [6]:
def objective(trial):
    
    ## Parameters to be evaluated
    param = dict(gamma = trial.suggest_float('gamma', 0.001, 100, log = True),
                 C = trial.suggest_float('C', 0.001, 100, log = True)
                 )

    
    scores = []

#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        
        print(fold, end = ' ')
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train , y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.fit_transform(X_valid)
        
        model = SVR(**param)
        model.fit(X_train, y_train)

        preds_valid = model.predict(X_valid)
        
        optR = OptimizedRounder()
        optR.fit(preds_valid, y_valid)
        coef = optR.coefficients()
        preds_valid = optR.predict(preds_valid, coef).astype(int)
        
        score = cohen_kappa_score(y_valid,  preds_valid, weights = "quadratic")
        scores.append(score)
        
    return np.mean(scores)

In [7]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 50, timeout = 3600)

[I 2023-02-07 05:23:48,789] A new study created in memory with name: no-name-f47ef7a9-5ee6-423d-8a87-d8ce8d2a77fa


0 1 2 3 4 

[I 2023-02-07 05:23:50,968] Trial 0 finished with value: 0.5323880393356217 and parameters: {'gamma': 0.005251583707631983, 'C': 44.366136196986396}. Best is trial 0 with value: 0.5323880393356217.


0 1 2 3 4 

[I 2023-02-07 05:23:53,100] Trial 1 finished with value: 0.5615095702383444 and parameters: {'gamma': 0.42024562480270605, 'C': 2.2711369345105425}. Best is trial 1 with value: 0.5615095702383444.


...

[I 2023-02-07 05:25:30,827] Trial 47 finished with value: 0.5111080659546994 and parameters: {'gamma': 2.703943350746687, 'C': 0.43770296647500184}. Best is trial 26 with value: 0.5660708973431483.


0 1 2 3 4 

[I 2023-02-07 05:25:32,889] Trial 48 finished with value: 0.5559021101583037 and parameters: {'gamma': 1.2011031019690146, 'C': 0.10833093440576959}. Best is trial 26 with value: 0.5660708973431483.


0 1 2 3 4 

[I 2023-02-07 05:25:34,956] Trial 49 finished with value: 0.558608542602665 and parameters: {'gamma': 0.40872137142354886, 'C': 1.8303947620833592}. Best is trial 26 with value: 0.5660708973431483.


# SVM Modeling

In [11]:
test_md.head()

,sulphate/density,alcohol_density,alcohol,sulphates
0,0.591491,9.775304,9.8,0.59
1,0.682827,9.958600,10.0,0.68
2,0.651472,9.478530,9.5,0.65
3,0.653398,11.639160,11.7,0.65
4,0.483111,12.717568,12.8,0.48


In [12]:
svm_cv_scores = list()
preds = list()

skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
for train_ix, test_ix in skf.split(X, Y):
        
    ## Splitting the data 
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    test_md = scaler.fit_transform(test_md)
    
    ## Building the model
    svm_md = SVR(gamma = 1.0542080231873707, C = 0.2771400815744156).fit(X_train, Y_train)
    
    ## Predicting on X_test and test
    svm_pred_1 = svm_md.predict(X_test)
    svm_pred_2 = svm_md.predict(test_md)
        
    ## Applying Optimal Rounder (using abhishek approach)
    optR = OptimizedRounder()
    optR.fit(svm_pred_1, Y_test)
    coef = optR.coefficients()
    svm_pred_1 = optR.predict(svm_pred_1, coef).astype(int)
    svm_pred_2 = optR.predict(svm_pred_2, coef).astype(int)
        
    ## Computing roc-auc score
    svm_cv_scores.append(cohen_kappa_score(Y_test, svm_pred_1, weights = 'quadratic'))
    preds.append(svm_pred_2)

svm_cv_score = np.mean(svm_cv_scores)    
print('The average weighted quadratic kappa score over 5-folds is:', svm_cv_score)

The average weighted quadratic kappa score over 5-folds is: 0.5660708973431483


In [13]:
svm_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ]

submission['quality'] = svm_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,5
2,2058,5
3,2059,6
4,2060,6


In [14]:
submission['quality'].value_counts()

5    748
6    418
7    206
Name: quality, dtype: int64

In [15]:
submission.to_csv('SVM_Reg_FE_1.csv', index = False)